# Deploy a Hyper-Segmented Model with TDStone2 in less than a minute
# Using Scikit Learn Regressor Pipeline

## 1 - Connect to Vantage

In [1]:
import teradataml as tdml
import json
import warnings
warnings.filterwarnings('ignore')
tdml.configure.byom_install_location = 'mldb'
tdml.__version__

'20.00.00.03'

In [2]:
with open('../connections/me.json','r') as f:
    Param_ = json.load(f)

with open('../connections/csae2.json','r') as f:
    Param = json.load(f)

with open('../connections/csae.json','r') as f:
    Param_ = json.load(f)

with open('../connections/vantage24.json','r') as f:
    Param_ = json.load(f)

tdml.create_context(**Param)

Engine(teradatasql://:***@sto-xo0saiv0ot1sxmj2.env.clearscape.teradata.com?DATABASE=demo_user&user=DEMO_USER)

## 2 - The Hyper-Segmented Dataset

In [3]:
dataset = tdml.DataFrame(tdml.in_schema(Param['database'],'dataset_00'))
dataset

Partition_ID,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,flag,Y1,Y2,FOLD
1,10,-1.3943636803788597,1.0442439732804276,0.5869266486642999,-0.5500792373351924,0.09460275826189339,-1.34775626852841,-0.7346483551701115,-0.3561456066099827,-0.7236299160771967,1,-0.3386557428971961,0,train
1,13,-0.5290938768944691,0.6903393570933072,1.1537722253491944,-0.5504227477456725,0.6151634836663831,0.8021212464260443,-0.6110152048039241,0.3207526094772286,0.14503663819412438,0,-1.0492098083949517,0,train
1,14,-0.598405175793809,0.8340686683287597,0.89004573457406,0.9819208743083102,0.5117475378578401,1.6287891032576154,-0.2755045333598663,1.7659255107743697,0.06473147359001329,1,-0.5724473668434706,0,train
1,15,0.2859261017119707,0.09716157967151284,2.349656188957245,-1.3021487580877988,-0.10565250962147228,-0.02334997181715255,1.6141339448390328,1.2815721200452719,-0.22501852672057815,0,-1.9343733053103433,0,train
1,18,0.2790094163338728,-0.5993945124563128,-0.33743210912394056,-0.568501591445928,0.39819264552127004,1.009874734630375,0.0026066375003983575,-0.6735609305438796,-0.14022970447284688,0,-0.4765066566867408,0,train
1,19,-0.4009889552430267,-0.18047857220268995,-1.029449128949099,0.6158182047188758,-0.15595135343233418,0.7665865621847803,-0.11608572643548498,-0.20567842971103983,1.198469649383455,0,0.8450125207381537,1,train
1,17,-0.5880929389594226,0.477107037715851,-0.2168553352166439,-0.05274077684634872,-0.46211667891337066,-0.311643246160771,-0.06351649783185169,1.2484455788348916,1.4893193936051612,1,-1.8571250105782788,0,train
1,11,-0.3059304495805793,1.3324949515193687,1.2540806720724382,-0.38630974657985606,0.26855053283294333,-0.4658146315919535,-0.5153015032790745,-1.0655645081450833,0.1298707397139366,0,-0.06712115310291952,0,train
1,9,0.09158721666417137,-0.6465926402549871,0.1784250667686505,0.1205870580710051,-1.319806353755415,-0.16685694055988687,0.5778869101906173,-0.3966612279520089,2.2176344103659003,1,-1.012849796211918,0,train
1,6,0.7351511655220212,-0.08227531133025473,0.5291482580000527,-0.2532613061965269,-0.13573158295215673,-0.9660524985979275,-1.362613795443117,0.2312326315018219,0.7547788732509687,1,-0.9613912453980786,0,train


In [4]:
summary = dataset.groupby('Partition_ID').count()
summary

Partition_ID,count_ID,count_X1,count_X2,count_X3,count_X4,count_X5,count_X6,count_X7,count_X8,count_X9,count_flag,count_Y1,count_Y2,count_FOLD
1,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
6,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
4,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
9,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000


In [5]:
summary.shape

(4, 15)

## 3 - Hyper-segmented model deployment

### 3.1 - Engineering of the scikit-learn classifier pipeline

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.pipeline import Pipeline

# Example usage
steps_anomaly_detection = [
    ('scaler', StandardScaler()),
    ('one_class_svm', OneClassSVM(
        kernel='rbf',  # Radial Basis Function Kernel
        nu=0.05,       # An upper bound on the fraction of training errors and a lower bound of the fraction of support vectors.
        gamma='auto'   # Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’. If ‘auto’, 1/n_features will be used.
    ))
]

### 3.2 - Deployment of the scikit-learn pipeline

In [7]:
from tdstone2.tdshypermodel import HyperModel
from tdstone2.tdstone import TDStone
sto = TDStone(schema_name = Param['database'], SEARCHUIFDBPATH = Param['user'])

In [8]:
model_parameters = {
    "column_categorical": ['flag'],
    "column_names_X": ['X1','X2','X3','X4','X5','X6','X7','X8','X9','flag']
}

In [9]:
%%time
model = HyperModel(tdstone            = sto,
                   metadata           = {'project': 'test'},
                   skl_pipeline_steps = steps_anomaly_detection,
                   model_parameters   = model_parameters,
                   dataset            = tdml.in_schema(Param['database'],'dataset_00'),
                   id_row             = 'ID',
                   id_partition       = 'Partition_ID',
                   id_fold            = 'FOLD',
                   fold_training      = 'train',
                   convert_to_onnx    = True
                  )

registration of mapper with id = fca98df7-3e26-41f5-a8a8-0036c348fb9a
creation of dedicated mapper table : demo_user.TDS_MAPPER_fca98df7_3e26_41f5_a8a8_0036c348fb9a
creation of the on clause view demo_user.TDS_ON_CLAUSE_TRAINING_fca98df7_3e26_41f5_a8a8_0036c348fb9a
creation of the sto view demo_user.TDS_STO_TRAINING_fca98df7_3e26_41f5_a8a8_0036c348fb9a
registration of mapper with id = 57d6b26b-7691-4de9-b465-1d48affe93a8
creation of dedicated mapper table : demo_user.TDS_MAPPER_57d6b26b_7691_4de9_b465_1d48affe93a8
creation of the on clause view demo_user.TDS_ON_CLAUSE_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8
creation of the volatile table on on clause view demo_user.TDS_STO_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8
creation of the sto view demo_user.TDS_STO_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8
register hyper model with id : 533fd2f6-cdf9-4c18-bfd1-11f6c017a587
hyper model : 533fd2f6-cdf9-4c18-bfd1-11f6c017a587
CPU times: total: 109 ms
Wall time: 28.2 s


In [10]:
sto.list_hyper_models()

CREATION_DATE,ID,ID_MODEL,ID_MAPPER_TRAINING,ID_MAPPER_SCORING,METADATA
2025-01-08 05:26:48.160000-05:,654b028b-741e-4a46-8e5d-d966ac8a07b1,1feb2a86-622c-4c56-8e94-4c367d1a232e,05842693-ddf1-4589-8d7b-a7ea4e952b95,a93604aa-2204-45a6-b4f5-1edab0414da1,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 06:28:41.130000-05:,533fd2f6-cdf9-4c18-bfd1-11f6c017a587,0fe42030-d412-411b-9695-8571c07507a0,fca98df7-3e26-41f5-a8a8-0036c348fb9a,57d6b26b-7691-4de9-b465-1d48affe93a8,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 05:36:52.330000-05:,91fb8620-0b80-4cec-a8a5-e1de531e608e,06e9bc9e-32f1-4f7b-88f9-5a09e1ac8204,72ef3ebe-73d5-4a13-89d3-e093b53c81d1,d14273b9-df63-444b-98ce-0bad17935b12,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 05:31:04.970000-05:,c5ef0032-dd20-4b48-817e-6fe7e3c42e55,505ea202-bbf4-4b2c-9639-2a18176d97f1,dc1cdd3e-111f-440e-ae0e-c0b3336c0716,431b1e0c-57a4-4ae8-bb3e-3a2372349857,"{""user"": ""dm250067"", ""project"": ""test""}"


In [11]:
id = sto.list_hyper_models()[['ID','CREATION_DATE']].to_pandas().reset_index().sort_values('CREATION_DATE', ascending=False).iloc[0,0]
id

'533fd2f6-cdf9-4c18-bfd1-11f6c017a587'

## 4 - Execution of the deployed hypermodel

### 4.1 - Models Training

In [12]:
%%time
model.train()

insert trained models in demo_user.TDS_TRAINED_MODELS_fca98df7_3e26_41f5_a8a8_0036c348fb9a
- access pickle models in demo_user.V_TDS_TRAINED_MODELS_fca98df7_3e26_41f5_a8a8_0036c348fb9a_PICKLE
- access  onnx  models in demo_user.V_TDS_TRAINED_MODELS_fca98df7_3e26_41f5_a8a8_0036c348fb9a_ONNX
- onnx byom catalog    in demo_user.V_TDS_TRAINED_MODELS_fca98df7_3e26_41f5_a8a8_0036c348fb9a_BYOM_CATALOG
CPU times: total: 0 ns
Wall time: 12.4 s


In [13]:
model.get_trained_models().groupby(['TD_TIMECODE','MODEL_TYPE']).count().sort(['TD_TIMECODE','MODEL_TYPE'],ascending=False)

demo_user TDS_MODEL_REPOSITORY


TD_TIMECODE,MODEL_TYPE,count_ID_PROCESS,count_Partition_ID,count_ID_MODEL,count_ID_TRAINED_MODEL,count_STATUS
9999-01-01 00:00:00.000000-04:,pickle,4,4,4,4,4
9999-01-01 00:00:00.000000-04:,onnx,4,4,4,4,4


### 4.2 - Model Scoring

In [14]:
%%time
model.score()

creation of the on clause view demo_user.TDS_ON_CLAUSE_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8
creation of the volatile table on on clause view demo_user.TDS_STO_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8
insert scores in demo_user.TDS_SCORES_57d6b26b_7691_4de9_b465_1d48affe93a8
CPU times: total: 93.8 ms
Wall time: 23 s


In [15]:
model.get_model_predictions().groupby('TD_TIMECODE').count()

demo_user TDS_SCORES_57d6b26b_7691_4de9_b465_1d48affe93a8


TD_TIMECODE,count_ID_PROCESS,count_ID_TRAINED_MODEL,count_Partition_ID,count_ID,count_anomaly_prediction,count_anomaly_score,count_decision_function
9999-01-01 00:00:00.000000-04:,40000,40000,40000,40000,40000,40000,40000


In [16]:
model.get_model_predictions()

demo_user TDS_SCORES_57d6b26b_7691_4de9_b465_1d48affe93a8


TD_TIMECODE,ID_PROCESS,ID_TRAINED_MODEL,Partition_ID,ID,anomaly_prediction,anomaly_score,decision_function
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8,303151d8-425c-478b-9e60-5034302e069c,4,28365,1.0,9.23391768625777,4.509748264786916
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8,303151d8-425c-478b-9e60-5034302e069c,4,25206,1.0,9.70329233096049,4.979122909489636
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8,1ecac149-c3c2-4f73-953e-cac0493ed72a,1,8496,1.0,7.770037660204008,3.097663240446253
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8,303151d8-425c-478b-9e60-5034302e069c,4,27313,1.0,6.006099905429239,1.2819304839583845
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8,303151d8-425c-478b-9e60-5034302e069c,4,24756,1.0,5.9121897193573805,1.188020297886526
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8,303151d8-425c-478b-9e60-5034302e069c,4,20697,1.0,6.319797705611916,1.5956282841410614
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8,7914e873-7d00-4c18-8506-951c2ff1342a,6,31268,1.0,6.75085965440556,1.9516674462262724
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8,7914e873-7d00-4c18-8506-951c2ff1342a,6,31812,1.0,9.497493993836265,4.6983017856569775
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8,1ecac149-c3c2-4f73-953e-cac0493ed72a,1,3644,1.0,8.801543039734602,4.129168619976847
9999-01-01 00:00:00.000000-04:,TDS_STO_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8,303151d8-425c-478b-9e60-5034302e069c,4,24306,1.0,9.436023685053652,4.711854263582797


## 5 - A bit of administration: Model Lineage

### 5.1 - Access to the list of deployed codes

In [17]:
sto.list_codes()

ID,CODE_TYPE,METADATA
48dea5f2-7862-4f1a-9b71-3f03661772e2,python class,"{""user"": ""dm250067"", ""code_type"": ""python class"", ""project"": ""test"", ""script_path"": ""from memory""}"
0a923c33-9369-407e-80a0-187ffa2a9837,python class,"{""user"": ""dm250067"", ""code_type"": ""python class"", ""project"": ""test"", ""script_path"": ""from memory""}"
c88da75f-4c35-4fb4-8b2b-52105610adf7,python class,"{""user"": ""dm250067"", ""code_type"": ""python class"", ""project"": ""test"", ""script_path"": ""from memory""}"
2498d459-7d97-46c3-8cec-40f350c7230f,python class,"{""user"": ""dm250067"", ""code_type"": ""python class"", ""project"": ""test"", ""script_path"": ""from memory""}"


### 5.2 - List of deployed models (code + parameters)

In [18]:
sto.list_models()

ID,ID_CODE,ARGUMENTS,METADATA
505ea202-bbf4-4b2c-9639-2a18176d97f1,0a923c33-9369-407e-80a0-187ffa2a9837,"{""sto_parameters"": {""columnnames"": [""Partition_ID"", ""ID"", ""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag"", ""Y1"", ""Y2"", ""FOLD""], ""float_columnames"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""Y1""], ""integer_columnames"": [""Partition_ID"", ""ID"", ""flag"", ""Y2""], ""category_columns"": [], ""output_format"": [""pickle""]}, ""model_parameters"": {""target"": ""Y1"", ""column_categorical"": [""flag""], ""column_names_X"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag""], ""arguments"": {""regressor__max_depth"": 5, ""regressor__n_estimators"": 95}}}","{""user"": ""dm250067"", ""project"": ""test""}"
06e9bc9e-32f1-4f7b-88f9-5a09e1ac8204,48dea5f2-7862-4f1a-9b71-3f03661772e2,"{""sto_parameters"": {""columnnames"": [""Partition_ID"", ""ID"", ""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag"", ""Y1"", ""Y2"", ""FOLD""], ""float_columnames"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""Y1""], ""integer_columnames"": [""Partition_ID"", ""ID"", ""flag"", ""Y2""], ""category_columns"": [], ""output_format"": [""pickle""]}, ""model_parameters"": {""column_categorical"": [""flag""], ""column_names_X"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag""], ""target"": """", ""arguments"": {""one_class_svm__gamma"": ""auto"", ""one_class_svm__nu"": 0.05}}}","{""user"": ""dm250067"", ""project"": ""test""}"
1feb2a86-622c-4c56-8e94-4c367d1a232e,c88da75f-4c35-4fb4-8b2b-52105610adf7,"{""sto_parameters"": {""columnnames"": [""Partition_ID"", ""ID"", ""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag"", ""Y1"", ""Y2"", ""FOLD""], ""float_columnames"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""Y1""], ""integer_columnames"": [""Partition_ID"", ""ID"", ""flag"", ""Y2""], ""category_columns"": [], ""output_format"": [""pickle""]}, ""model_parameters"": {""target"": ""Y2"", ""column_categorical"": [""flag"", ""Y2""], ""column_names_X"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag""], ""arguments"": {""classifier__max_depth"": 5, ""classifier__n_estimators"": 95}}}","{""user"": ""dm250067"", ""project"": ""test""}"
0fe42030-d412-411b-9695-8571c07507a0,2498d459-7d97-46c3-8cec-40f350c7230f,"{""sto_parameters"": {""columnnames"": [""Partition_ID"", ""ID"", ""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag"", ""Y1"", ""Y2"", ""FOLD""], ""float_columnames"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""Y1""], ""integer_columnames"": [""Partition_ID"", ""ID"", ""flag"", ""Y2""], ""category_columns"": [], ""output_format"": [""onnx"", ""pickle""]}, ""model_parameters"": {""column_categorical"": [""flag""], ""column_names_X"": [""X1"", ""X2"", ""X3"", ""X4"", ""X5"", ""X6"", ""X7"", ""X8"", ""X9"", ""flag""], ""target"": """", ""arguments"": {""one_class_svm__gamma"": ""auto"", ""one_class_svm__nu"": 0.05}}}","{""user"": ""dm250067"", ""project"": ""test""}"


### 5.3 - List of available mappers (mapping between partitions and models or trained models)

In [19]:
sto.list_mappers()

ID,MAPPER_TYPE,TABLE_NAME,CODE_REPOSITORY,MODEL_REPOSITORY,TRAINED_MODEL_REPOSITORY,FEATURE_REPOSITORY,SCORES_REPOSITORY,DATASET_OBJECT,COL_ID_ROW,COL_ID_PARTITION,COL_FOLD,ON_CLAUSE_VIEW,STO_VIEW,METADATA
72ef3ebe-73d5-4a13-89d3-e093b53c81d1,training,TDS_MAPPER_72ef3ebe_73d5_4a13_89d3_e093b53c81d1,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_72ef3ebe_73d5_4a13_89d3_e093b53c81d1,None,None,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_TRAINING_72ef3ebe_73d5_4a13_89d3_e093b53c81d1,TDS_STO_TRAINING_72ef3ebe_73d5_4a13_89d3_e093b53c81d1,"{""user"": ""dm250067""}"
d14273b9-df63-444b-98ce-0bad17935b12,scoring,TDS_MAPPER_d14273b9_df63_444b_98ce_0bad17935b12,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_72ef3ebe_73d5_4a13_89d3_e093b53c81d1,None,TDS_SCORES_d14273b9_df63_444b_98ce_0bad17935b12,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_SCORING_d14273b9_df63_444b_98ce_0bad17935b12,TDS_STO_SCORING_d14273b9_df63_444b_98ce_0bad17935b12,"{""user"": ""dm250067""}"
fca98df7-3e26-41f5-a8a8-0036c348fb9a,training,TDS_MAPPER_fca98df7_3e26_41f5_a8a8_0036c348fb9a,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_fca98df7_3e26_41f5_a8a8_0036c348fb9a,None,None,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_TRAINING_fca98df7_3e26_41f5_a8a8_0036c348fb9a,TDS_STO_TRAINING_fca98df7_3e26_41f5_a8a8_0036c348fb9a,"{""user"": ""dm250067""}"
431b1e0c-57a4-4ae8-bb3e-3a2372349857,scoring,TDS_MAPPER_431b1e0c_57a4_4ae8_bb3e_3a2372349857,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_dc1cdd3e_111f_440e_ae0e_c0b3336c0716,None,TDS_SCORES_431b1e0c_57a4_4ae8_bb3e_3a2372349857,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,TDS_STO_SCORING_431b1e0c_57a4_4ae8_bb3e_3a2372349857,"{""user"": ""dm250067""}"
a93604aa-2204-45a6-b4f5-1edab0414da1,scoring,TDS_MAPPER_a93604aa_2204_45a6_b4f5_1edab0414da1,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_05842693_ddf1_4589_8d7b_a7ea4e952b95,None,TDS_SCORES_a93604aa_2204_45a6_b4f5_1edab0414da1,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_SCORING_a93604aa_2204_45a6_b4f5_1edab0414da1,TDS_STO_SCORING_a93604aa_2204_45a6_b4f5_1edab0414da1,"{""user"": ""dm250067""}"
05842693-ddf1-4589-8d7b-a7ea4e952b95,training,TDS_MAPPER_05842693_ddf1_4589_8d7b_a7ea4e952b95,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_05842693_ddf1_4589_8d7b_a7ea4e952b95,None,None,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_TRAINING_05842693_ddf1_4589_8d7b_a7ea4e952b95,TDS_STO_TRAINING_05842693_ddf1_4589_8d7b_a7ea4e952b95,"{""user"": ""dm250067""}"
dc1cdd3e-111f-440e-ae0e-c0b3336c0716,training,TDS_MAPPER_dc1cdd3e_111f_440e_ae0e_c0b3336c0716,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_dc1cdd3e_111f_440e_ae0e_c0b3336c0716,None,None,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_TRAINING_dc1cdd3e_111f_440e_ae0e_c0b3336c0716,TDS_STO_TRAINING_dc1cdd3e_111f_440e_ae0e_c0b3336c0716,"{""user"": ""dm250067""}"
57d6b26b-7691-4de9-b465-1d48affe93a8,scoring,TDS_MAPPER_57d6b26b_7691_4de9_b465_1d48affe93a8,TDS_CODE_REPOSITORY,TDS_MODEL_REPOSITORY,TDS_TRAINED_MODELS_fca98df7_3e26_41f5_a8a8_0036c348fb9a,None,TDS_SCORES_57d6b26b_7691_4de9_b465_1d48affe93a8,"""demo_user"".""dataset_00""",ID,Partition_ID,FOLD,TDS_ON_CLAUSE_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8,TDS_STO_SCORING_57d6b26b_7691_4de9_b465_1d48affe93a8,"{""user"": ""dm250067""}"


### 5.4 - List of Hypermodels ( models and mappers mapping)

In [20]:
sto.list_hyper_models()

CREATION_DATE,ID,ID_MODEL,ID_MAPPER_TRAINING,ID_MAPPER_SCORING,METADATA
2025-01-08 05:36:52.330000-05:,91fb8620-0b80-4cec-a8a5-e1de531e608e,06e9bc9e-32f1-4f7b-88f9-5a09e1ac8204,72ef3ebe-73d5-4a13-89d3-e093b53c81d1,d14273b9-df63-444b-98ce-0bad17935b12,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 05:26:48.160000-05:,654b028b-741e-4a46-8e5d-d966ac8a07b1,1feb2a86-622c-4c56-8e94-4c367d1a232e,05842693-ddf1-4589-8d7b-a7ea4e952b95,a93604aa-2204-45a6-b4f5-1edab0414da1,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 05:31:04.970000-05:,c5ef0032-dd20-4b48-817e-6fe7e3c42e55,505ea202-bbf4-4b2c-9639-2a18176d97f1,dc1cdd3e-111f-440e-ae0e-c0b3336c0716,431b1e0c-57a4-4ae8-bb3e-3a2372349857,"{""user"": ""dm250067"", ""project"": ""test""}"
2025-01-08 06:28:41.130000-05:,533fd2f6-cdf9-4c18-bfd1-11f6c017a587,0fe42030-d412-411b-9695-8571c07507a0,fca98df7-3e26-41f5-a8a8-0036c348fb9a,57d6b26b-7691-4de9-b465-1d48affe93a8,"{""user"": ""dm250067"", ""project"": ""test""}"


## 6 - Use BYOM with ONNX models

In [21]:
onnx_catalog = model.get_byom_catalog()
onnx_catalog

demo_user V_TDS_TRAINED_MODELS_fca98df7_3e26_41f5_a8a8_0036c348fb9a_BYOM_CATALOG


model_id,model,Partition_ID
7914e873-7d00-4c18-8506-951c2ff1342a,b'8071208736B6C326F...',6
69a6e8d3-4e81-4fce-8cdf-95c4a56e4ce7,b'8071208736B6C326F...',9
303151d8-425c-478b-9e60-5034302e069c,b'8071208736B6C326F...',4
1ecac149-c3c2-4f73-953e-cac0493ed72a,b'8071208736B6C326F...',1


In [22]:
onnx_catalog_local = onnx_catalog[['model_id','Partition_ID']].to_pandas(num_rows=1)
onnx_catalog_local

,model_id,Partition_ID
0,7914e873-7d00-4c18-8506-951c2ff1342a,6


In [23]:
Partition_ID     = onnx_catalog_local.Partition_ID.values[0]
model_id         = onnx_catalog_local.model_id.values[0]

In [24]:
predictions_ = tdml.ONNXPredict(
    modeldata               = onnx_catalog[onnx_catalog.Partition_ID == Partition_ID],
    newdata                 = dataset[dataset.Partition_ID == Partition_ID],
    accumulate              = ['Partition_ID', 'ID'],
    overwrite_cached_models = '*',
)

In [25]:
predictions_.result

Partition_ID,ID,json_report
6,30010,"{""scores"":[[2.2668543]],""label"":[[1]]}"
6,30013,"{""scores"":[[1.169498]],""label"":[[1]]}"
6,30014,"{""scores"":[[5.387825]],""label"":[[1]]}"
6,30015,"{""scores"":[[0.572155]],""label"":[[1]]}"
6,30018,"{""scores"":[[1.3546047]],""label"":[[1]]}"
6,30019,"{""scores"":[[6.316168]],""label"":[[1]]}"
6,30017,"{""scores"":[[2.3828354]],""label"":[[1]]}"
6,30011,"{""scores"":[[4.159606]],""label"":[[1]]}"
6,30009,"{""scores"":[[-3.0517578E-4]],""label"":[[-1]]}"
6,30006,"{""scores"":[[3.2149105]],""label"":[[1]]}"


In [26]:
tdml.remove_context()

True